In [80]:
import cv2
from pathlib import Path
import cv_exp.basic
import cv_exp.basic as cv
import cv_exp.pupil_detection as pupil_detection
import cv_exp.draw as draw
import cv_exp.log as log
import cv_exp.take as take
import numpy as np
import glob
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load Specific Tf Model from its path
model_face_detection = '/home/roopesh/Desktop/MediaPipe/cv-exp-framework-master/data/models/face_detection_front.tflite'
model_face_landmarks ='/home/roopesh/Desktop/MediaPipe/cv-exp-framework-master/data/models/face_landmark.tflite'
model_iris_landmarks = '/home/roopesh/Desktop/MediaPipe/cv-exp-framework-master/data/models/iris_landmark.tflite'


iris_detector = pupil_detection.IrisDetectorMP(model_face_detection_path=model_face_detection,
                                                   model_face_landmarks_path=model_face_landmarks,
                                                   model_iris_landmarks_path=model_iris_landmarks)

import os
# (path):
files_dir ='/home/roopesh/Desktop/HSL/sample/Blue_Iris_Training_Image/'

files = os.listdir(files_dir)
sorted_images = sorted(files, key=lambda x : int(x[5:-4])) # Sorting the images in ascending order based on the number in the name of the image(Image123.jpg)...here(int x[5:-4] omit the (Image and .jpg)in file name-> it counts only in between number(123) and sort the images in ascending order)
for idx, file in enumerate(sorted_images):
    if file.lower().endswith(('.png', '.jpg')):
        image_path = files_dir + '/' + file

        image= cv2.imread(image_path)

# conversion numpy array into rgb image to show

        image_bgr_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_rgb_bgr = cv2.cvtColor(image_bgr_rgb, cv2.COLOR_RGB2BGR)
        out = image_rgb_bgr.copy()
        out_pupile = image.copy()
        detected_pupils, result = iris_detector.detect(image_rgb_bgr)

        if detected_pupils.is_ok():
            out_pupiles = draw.draw_pupils(out_pupile, detected_pupils.left, detected_pupils.right, color_center= (0,0,0), color_contour= (0,0,0))

        #print(detected_left)
        if result is not None:
            (face_detection, face_keypoints, face_landmarks,
                left_iris_landmarks, left_eyelid_landmarks,
                right_iris_landmarks, right_eyelid_landmarks,
                left_iris_segmentation, right_iris_segmentation,
                concat_landmarks) = result
            out = draw.draw_contour(out, left_iris_segmentation, thickness=1, color=(255, 255, 255))
            out = draw.draw_contour(out, right_iris_segmentation, thickness=1, color=(255, 255, 255))
#             print( "Left Iris Landmarks: ", left_iris_landmarks)

# Extraction Points From LEFT IRIS Landmarks:

# Extracting 5 Iris Points from Left iris :

            left_iris_points = [(point.x, point.y) for point in left_iris_landmarks]

            center_point_LeftIris=(np.round(left_iris_points[0]))
            for left_x,left_y in [center_point_LeftIris]:
                 lx, ly = left_x, left_y

            left_point_LeftIris=(np.round(left_iris_points[1]))
            for leftIris_left_x,LeftIris_left_y in [left_point_LeftIris]:
                LIL_x, LIL_y = leftIris_left_x, LeftIris_left_y

            right_point_LeftIris=(np.round(left_iris_points[3]))
            for rightIris_x,rightIris_y in [right_point_LeftIris]:
                RIL_x, RIL_y = rightIris_x, rightIris_y

            Top_point_LeftIris=(np.round(left_iris_points[2]))
            for TopIris_x,TopIris_y in [Top_point_LeftIris]:
                TI_x, TI_y = TopIris_x, TopIris_y

            Bottom_point_LeftIris=(np.round(left_iris_points[4]))
            for BottomIris_x,BottomIris_y in [Bottom_point_LeftIris]:
                BI_x, BI_y = BottomIris_x, BottomIris_y

# Distance Formula to calculate the distance between center to left_Iris_ points to get radius of iris
            distance_left_iris_center = np.sqrt((lx - LIL_x) ** 2 + (ly - LIL_y) ** 2)

# Distance Formula to calculate the distance between center to right_Iris_ points to get radius of iris
            distance_left_iris_center1 = np.sqrt((lx - RIL_x) ** 2 + (ly - RIL_y) ** 2)
    
    
    

# Circular mask
            mask = np.zeros(out_pupiles.shape, dtype=np.uint8) # Result from Mediapipe added here.

            center= (int(left_x),int(left_y))
            radius = (int((distance_left_iris_center)*0.80))-(int((distance_left_iris_center)*0.25))
            
            left_mask = cv2.circle(mask, center, radius,[255,255,255], -1 )

            result = cv2.bitwise_and(out_pupiles, left_mask)

# adding background Color- white
            result[mask==0] = 255 # Optional
           
            cv2.imwrite(f"/home/roopesh/Desktop/HSL/sample/Original_Mask_Mediapipe/Original_Mask/{file}.jpg",result)
           
        
# cropping iris area from the masked image
            x1= (RIL_x)+7  # x-axis --Landmarks points from top- right point from left iris
            y1= (TI_y)+7  # y-axis --Landmarks points from top_ left of left iris
            x=int(x1)
            y=int(y1)
           
          
            h= int(radius*2.35)-6  # height --Radius of iris times 2
            w=int(radius*2.35)-2   # width  --Radius of iris times 2
            crop = result[y:y+h, x:x+w]

            cv2.imwrite(f"/home/roopesh/Desktop/HSL/sample/Mask/Blue_Iris_Mask/{file}.jpg",crop)     
            
            
            
            # Circular mask
            mask1 = np.zeros(out_pupiles.shape, dtype=np.uint8) # Result from Mediapipe added here.

            center1= (int(left_x),int(left_y))
            radius1 = (int((distance_left_iris_center)*0.25))
            left_mask1 = cv2.circle(mask1, center1, radius1,(255,255,255), -1 )

            result1 = cv2.bitwise_and(out_pupiles, left_mask1)

# # adding background Color- white
            result1[mask1==0] = 255 # Optional
           
            cv2.imwrite(f"/home/roopesh/Desktop/HSL/sample/Original_Mask_Mediapipe/New_Mask/{file}.jpg",result1)
           
  
            
    

print('MediaPipe Processing Done')






MediaPipe Processing Done


In [ ]:

# Histogram for BGR iMAGES

            b, g, r = crop[:,:,0], crop[:,:,1], crop[:,:,2]
            hist_b = cv2.calcHist([b],[0],None,[255],[0,256])
            hist_g = cv2.calcHist([g],[0],None,[255],[0,256])
            hist_r = cv2.calcHist([r],[0],None,[255],[0,256])
            plt.plot(hist_b, color='b', label="B")
            plt.plot(hist_g, color='g', label="G")
            plt.plot(hist_r, color='r', label="R")
            plt.title("BGR Histogram for image: " + str(file[:-4]))
            plt.xlabel("Bins")
            plt.ylabel("# of Pixels")
            plt.legend()
            savefile = (str(file) + '.jpg')
       
            plt.savefig(f"/home/roopesh/Desktop/HSL/sample/Histogram/Histogram_BGR/Histogram_{savefile}")   # Brown
            plt.close(plt.gcf())

# Removing NOISE from the cropped image (<=10,  >=240) in all channel

            remove= np.array(crop)
            mask=np.all(remove<=[10,10,10], axis=2)
            remove[mask]=[0,0,0]
            mask=np.all(remove>=[230,230,230],axis=2)
            remove[mask]=[0,0,0]
            # mask=np.all(remove ==[255,255,255],axis=2)
            # remove[mask]=[0,0,0]
            mask=np.all(remove ==[0,255,255],axis=2)
            remove[mask]=[0,0,0]
            
            cv2.imwrite(f"/home/roopesh/Desktop/HSL/sample/Mask/Noise_Removed_Brown/{file}.jpg",remove)   # Brown
            
#  Histogram after NOISE REMOVED  cropped IMAGES

            b2, g2, r2 = remove[:,:,0], remove[:,:,1], remove[:,:,2]
            hist_b2 = cv2.calcHist([b2],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            hist_g2 = cv2.calcHist([g2],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            hist_r2 = cv2.calcHist([r2],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            plt.plot(hist_b2, color='b', label="B")
            plt.plot(hist_g2, color='g', label="G")
            plt.plot(hist_r2, color='r', label="R")
            plt.title("BGR Histogram for image: " + str(file[:-4]))
            plt.xlabel("Bins")
            plt.ylabel("# of Pixels")
            plt.legend()
            savefile = ( str(file) + '.jpg' )
            
            plt.savefig(f"/home/roopesh/Desktop/HSL/sample/Histogram/Noise_Removed_Histogram_BGR/Histogram_{savefile}")     # Brown
            plt.close(plt.gcf())
            
            
            
            
            # Extra Circular MASK: FOR PUPIL (Center Core)
            mask1 = np.zeros(out_pupiles.shape, dtype=np.uint8) # Result from Mediapipe added here.

# Circular mask landmarks for Left_iris

            center1= (int(left_x),int(left_y))
            radius1 = (int((distance_left_iris_center)/2))
            left_mask = cv2.circle(mask1, center1, radius1,(255,255,255), -1 )

# Creating Masked Image for Left Iris
            result1 = cv2.bitwise_and(out_pupiles, left_mask)

# adding background Color- white
            result1[mask1==0] = 255 # Optional

            x= int(RIL_x)+1   # x-axis --Landmarks points from top- right point from left iris
            y= int(TI_y)+1    # y-axis --Landmarks points from top_ left of left iris
            h= int(radius*2)  # height --Radius of iris times 2
            w=int(radius*2)   # width  --Radius of iris times 2
            crop1 = result1[y:y+h, x:x+w]

# saving extra circular mask + cropped of iris area

            cv2.imwrite(f"/home/roopesh/Desktop/HSL/sample/Original_Mask_Mediapipe/New_Angular_Mask/{file}.jpg",crop1)
          

# Image Subtraction for circular mask and extra circular mask

            # subtracted_image= cv2.subtract(crop, crop1)
            subtracted_image=(crop-crop1)

            cv2.imwrite(f"/home/roopesh/Desktop/HSL/sample/Mask/Subtracted_Image/{file}.jpg",subtracted_image)



# subtracted_image HIstogram

            b3, g3, r3 = subtracted_image[:,:,0], subtracted_image[:,:,1], subtracted_image[:,:,2]
            hist_b3 = cv2.calcHist([b3],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            hist_g3= cv2.calcHist([g3],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            hist_r3 = cv2.calcHist([r3],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            plt.plot(hist_b3, color='b', label="B")
            plt.plot(hist_g3, color='g', label="G")
            plt.plot(hist_r3, color='r', label="R")
            plt.title("BGR Histogram for image: " + str(file[:-4]))
            plt.xlabel("Bins")
            plt.ylabel("# of Pixels")
            plt.legend()
            savefile = ( str(file) + '.jpg' )
           
            plt.savefig(f"/home/roopesh/Desktop/HSL/sample/Histogram/Subtracted_Image_Histogram_BGR/Histogram_{savefile}")     # Brown
            plt.close(plt.gcf())

# Removing NOISE from the Subtracted  image (<=10,  >=240) in all channel

            remove1= np.array(subtracted_image)
            mask=np.all(remove1<=[10,10,10], axis=2)
            remove1[mask]=[0,0,0]
            mask=np.all(remove1>=[230,230,230],axis=2)
            remove1[mask]=[0,0,0]
            mask=np.all(remove1 ==[255,255,255],axis=2)
            remove1[mask]=[0,0,0]
            mask=np.all(remove1 ==[0,255,255],axis=2)
            remove1[mask]=[0,0,0]
            mask=np.all(remove1 ==[51,51,255],axis=2)
            remove1[mask]=[0,0,0]
            mask=np.all(remove1 ==[0,0,255],axis=2)
            remove1[mask]=[0,0,0]
            mask=np.all(remove1 ==[0,255,0],axis=2) #  Green
            remove1[mask]=[0,0,0]


            low_green=np.array([25,52,72])
            high_green=np.array([102,255,255])
            mask=cv2.inRange(remove1,low_green,high_green)

            mask=np.all(remove1 ==[0,0,0],axis=2)

        
            cv2.imwrite(f"/home/roopesh/Desktop/HSL/sample/Mask/Noise_Removed_Subtracted_Image/{file}.jpg",remove1)   # Brown


# Noise Removed subtracted_image HIstogram

            b4, g4, r4 = remove1[:,:,0], remove1[:,:,1], remove1[:,:,2]
            hist_b4 = cv2.calcHist([b4],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            hist_g4= cv2.calcHist([g4],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            hist_r4 = cv2.calcHist([r4],[0],None,[255],[1,256])  # [1,256] avoiding Black pixels
            plt.plot(hist_b4, color='b', label="B")
            plt.plot(hist_g4, color='g', label="G")
            plt.plot(hist_r4, color='r', label="R")
            plt.title("BGR Histogram for image : " + str(file[:-4]))
            plt.xlabel("Bins")
            plt.ylabel("# of Pixels")
            plt.legend()
            savefile = ( str(file) + '.jpg' )

            plt.savefig(f"/home/roopesh/Desktop/HSL/sample/Histogram/Noise_Removed_Subtracted_Image_Histogram_BGR/Histogram_{savefile}")     # Brown
            
            plt.close(plt.gcf())






# IMSHOW
#             cv2.imshow(f'image{file}', out )
            # cv2.imshow(f'image_pupile{file}', out_pupiles)
            # cv2.imshow(f'image_iris{file}', crop)
            # cv2.imshow(f'image_iris_noise_removed{file}', remove)
            # cv2.waitKey(6500)
            # cv2.destroyAllWindows()

# # save result of Mask. Give ur Directory path